# Multiple passes of laser through crystal

_____

#### Imports

In [ ]:
import sys, time
import math
import numpy as np
from pykern import pkcli
from pykern.pkcollections import PKDict

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rslaser.pulse import pulse
from rslaser.optics import element
from rslaser.optics import drift
from rslaser.optics import crystal

import scipy.constants as const

import srwlib
from srwlib import srwl

# 2D plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline

### Crystal and Laser Initialization

In [ ]:
params = PKDict(
        photon_e_ev=1.5498, # Photon energy [eV], calculated from 800nm wavelength
        nslice = 50,
        pulseE      = 1.0e-6,
        tau_fwhm    = 300.0e-12 / math.sqrt(2.),
        sigx_waist  = 1.2e-3 /1.18,
        sigy_waist  = 1.2e-3 /1.18,
)

num_slices = 20
crystal_params = PKDict(
    length = 2.5/100.0,  # [m]
    nslice = num_slices,
    n0          = [1.76 for _ in range(num_slices)],
    n2          = [16.0 for _ in range(num_slices)],
    pump_energy = 0.035,  # [J]
    pump_waist  = 1.2e-3 /1.18,  # [m]
)

prop_type = 'n0n2_srw'
gain = 1

num_propagate = 100

# Maximum value is num_propagate
prop_array = np.array([0, num_propagate])

## Propagation

In [ ]:
e_crystal = crystal.Crystal(crystal_params)
nslices_crystal = len(e_crystal.slice)

# Assume all crystal slices have same cell volume
dx = (e_crystal.slice[0].pop_inversion_xfin-e_crystal.slice[0].pop_inversion_xstart)/e_crystal.slice[0].pop_inversion_nx
dy = (e_crystal.slice[0].pop_inversion_yfin-e_crystal.slice[0].pop_inversion_ystart)/e_crystal.slice[0].pop_inversion_ny
cell_volume = dx*dy*e_crystal.slice[0].length

thisPulse = pulse.LaserPulse(params)
nslices_pulse = len(thisPulse.slice)

L_drift = 0.50  # [m]
e_drift = drift.Drift(L_drift)

photons_in_slice = np.zeros((nslices_pulse, num_propagate + 1))
total_n_excited_states = np.zeros(num_propagate + 1)

print('\nPropagation Type: ', prop_type, '\nGain: ', gain, '\n')
for prop_index in np.arange(num_propagate + 1):
    
    for laser_index in np.arange(nslices_pulse):
        photons_in_slice[laser_index,prop_index] = np.sum(np.sum(thisPulse.slice[laser_index].n_photons_2d.mesh))
    
    for crystal_index in np.arange(nslices_crystal):
        total_n_excited_states[prop_index] += np.sum(np.sum(e_crystal.slice[crystal_index].pop_inversion_mesh * cell_volume))

    if (prop_index < num_propagate):
        #print('\nPropagating ', nslices_pulse, 'laser slices through ', nslices_crystal,' crystal slices')
        thisPulse = e_crystal.propagate(thisPulse, prop_type, calc_gain=gain)
        thisPulse = e_drift.propagate(thisPulse, 'default')

# Assume all pulse slices have same length
fig_x = thisPulse.slice[0].ds *(np.arange(nslices_pulse) + 0.5)

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
for prop_value in prop_array:
    plot_label = str(prop_value)+' propagations'
    plt.plot(fig_x,photons_in_slice[:,prop_value], label=plot_label)
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.1, box.width, box.height * 0.9])
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), fancybox=True, shadow=True, ncol=5)
ax.tick_params(direction="in")
ax.set_ylabel(r'Total Number of Photons')
ax.set_xlabel(r'Distance from Laser Front [m]')
plt.title('Longitudinal Photon Distribution')

In [ ]:
wfr_sim_final=thisPulse.slice[0].wfr

x_sim_final=np.linspace(wfr_sim_final.mesh.xStart,wfr_sim_final.mesh.xFin,wfr_sim_final.mesh.nx)
y_sim_final=np.linspace(wfr_sim_final.mesh.yStart,wfr_sim_final.mesh.yFin,wfr_sim_final.mesh.ny)

e_total = thisPulse.extract_total_2d_elec_fields()
intens_2d_sim_final = 0.5 *const.c *const.epsilon_0 *(e_total.re**2.0 + e_total.im**2.0)
    
with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(6,3.6))
    ax = fig.gca()
    plt.pcolormesh(y_sim_final*(1e3), x_sim_final*(1e3), intens_2d_sim_final, cmap=plt.cm.viridis, shading='auto') #viridis
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (100 propagations)')

In [ ]:
intensity_2d = intens_2d_sim_final
max_intensity = np.max(intensity_2d)

center_x = np.where(intensity_2d == max_intensity)[0][0]
center_y = np.where(intensity_2d == max_intensity)[1][0]
keep_indices = np.where(intensity_2d >= 0.01*max_intensity)
distance = math.floor(np.max(np.sqrt((keep_indices[0] -center_x)**2.0 + (keep_indices[1] -center_y)**2.0)))
#distance = math.ceil(np.max(np.sqrt((keep_indices[0] -center_x)**2.0 + (keep_indices[1] -center_y)**2.0)))
        
phase_sim_final = srwlib.array('d', [0]*wfr_sim_final.mesh.nx*wfr_sim_final.mesh.ny) # "flat" array to take 2D phase data
srwl.CalcIntFromElecField(phase_sim_final, wfr_sim_final, 0, 4, 3, wfr_sim_final.mesh.eStart, 0, 0) #extracts the phase; must use double precision

# Reshaping data from flat to 2D array
phase_2d_sim_final = np.array(phase_sim_final).reshape((wfr_sim_final.mesh.nx, wfr_sim_final.mesh.ny), order='C').astype(np.float64)

phase_2d = phase_2d_sim_final
phase_2d_cut = phase_2d[center_x-distance:center_x+distance,center_y-distance:center_y+distance]
phase_2d_new = np.pad(phase_2d_cut, (int((phase_2d.shape[0] - phase_2d_cut.shape[0])/2),
                                     int((phase_2d.shape[1] - phase_2d_cut.shape[1])/2)),
                      mode='constant')

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(6,3.6))
    ax = fig.gca()
    plt.pcolormesh(y_sim_final*(1e3), x_sim_final*(1e3), phase_2d_new, cmap=plt.cm.viridis, shading='auto') #np.unwrap(phase_2d_sim_final)
    plt.colorbar()
    # plt.xlim(-1.95,1.95)
    # plt.ylim(-1.95,1.95)
    ax.set_ylabel(r'Vertical Position [mm]')#[mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')#[mm]')
    ax.set_title('Phase (100 propagations)')

## Comparison Plot

In [ ]:
fig, ax1 = plt.subplots(figsize=(5,3))
ax2 = ax1.twinx()
ax1.plot(fig_x,photons_in_slice[:,0], 'k', label='Initialization')
ax2.plot(fig_x,photons_in_slice[:,num_propagate], '--r', label='{} Passes Through Crystal'.format(num_propagate))
fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.01), fancybox=True, shadow=True, ncol=5)
ax1.tick_params(direction="in")
ax2.tick_params(direction="in")
ax1.set_xlabel(r'Distance from Laser Front [m]')
ax1.set_ylabel(r'Number of Photons, Initialized')
ax2.set_ylabel(r'Number of Photons, Propagated',color='r')
plt.title('Longitudinal Photon Distribution')

In [ ]:
total_n_photons = np.sum(photons_in_slice, axis=0)

fig, ax1 = plt.subplots(figsize=(5,3))
ax2 = ax1.twinx()
ax1.plot(np.arange(num_propagate+1),total_n_photons, 'k', label='Photons')
ax2.plot(np.arange(num_propagate+1),total_n_excited_states, 'r', label='Excited States')
fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.01), fancybox=True, shadow=True, ncol=5)
ax1.tick_params(direction="in")
ax2.tick_params(direction="in")
ax1.set_xlabel(r'Number of Propagations')
ax1.set_ylabel(r'Number of Photons')
ax2.set_ylabel(r'Number of Excited States',color='r')
#plt.title('Longitudinal Photon Distribution')

plt.figure(figsize=(5,3))
plt.plot(np.arange(num_propagate+1),total_n_photons, '--k', label='Photons')
plt.plot(np.arange(num_propagate+1),total_n_excited_states, ':k', label='Excited States')
plt.plot(np.arange(num_propagate+1),total_n_photons+total_n_excited_states,'k', label='Total')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=True, ncol=5)
plt.tick_params(direction="in")
plt.xlabel(r'Number of Propagations')
plt.title(r'Change in Photons and Excited States')
#plt.ylabel(r'Number of Photons or Excited States')